# Filling in the Gaps

You're probably still a little confused about a few things from the lesson:

* Why does `print(bytes([10, 100, 200])) -> b'\nd\xc8'` looks so odd? 
    * What's with the `\n`? 
    * What's with the `\x`? 
    * Are the `c` and the `8` related to the `\x`?
    * Is `d` related to either `\n`, `\x`, or neither?
* How should we interpret these exotic byte literals when you encounter one _in the wild_?
* How does the magical `int.from_bytes` function work?
* How does its inverse function `int.to_bytes` work?

To answer these questions we will make a `bites` class that acts just like `bytes`, but with a few minor differences: 
* Prints itself with strings (`"\x00\xff"`) instead of instead of the special `bytes` literals (`b"\x00\xff"`) .
* Will duplicate the functionality of the magical `int.from_bytes` and `int.to_bytes` methods by implementing its `bites.from_int` and `bites.to_int` methods.

Here's a skeleton of the class:

In [71]:
class bites:
    
    def __init__(self, ...):
        ...
        
    def __repr__(self):
        return "(cheating) " + repr(self.values)

    def to_int(self, byte_order):
        raise NotImplementedError()
    
    @classmethod
    def from_int(cls, n, length, byte_order):
        raise NotImplementedError()

SyntaxError: invalid syntax (<ipython-input-71-db365cdeb256>, line 3)

# Representation & Numbers

A "byte" is a value `x` such that `0 <= x < 256`.

Simple.

How can we represent a number above 256 as a series of bytes? Hmm ...

Let's think about how we solve this with our normal decimal numbers. Instead of 256 separate values or "places", we only have 10 places with values `y` such that `0 <= y < 9`.

What happens when we encounter a number outside this range? Say the number after 9, `9 + 1`? We no longer have a "numberal" or number symbol to represent it. We could invent one of course, but that would be a memorization nightmare ...

Instead we employ an ingenius trick to utilize _two numerals to rempresent 1 number_. It's really clever when you think of it. Of course, we write this `9 + 1` as `10` -- the left character represents that we have run out of numerals 1 time. It's the 10's place. The right numberal says that we haven't started going up again. We can repeat this procedure indefinitely. Here's the number 518:

<center>

```
     0              5           1         8
----------  ...  ---------- ---------- ----------
  10**n's          100's       10's       1's  

``` 
</center>

Therefore, any number `N` can be represented as a sum


$$ \sum_{i=0}^{i=n} a_{i} *  10^i = a_n *  10^n + ... + a_2 *  10^2 + a_1 *  10^1 + a_{0} *  10^0 $$


We can employ exactly the same trick to represent numbers greater than 256 as a series of bytes (numbers 0 <= x < 256). Just replaces the old threshold 10 with the new threshold 256:

$$ \sum_{i=0}^{i=n} a_{i} * 256^i =  a_n * 256^n + ... + a_2 * 256^2 + a_1 * 256^1 + a_{0} * 256^0 $$

The trick here is to go from an `int` to a list of values where each item represents a place in the base 256 expansion of the `int` we're dealing with. Here's the number 518 (2 * 256^1 + 6 * 256^0):

<center>
    
```
   0                0           2         6
----------  ...  ---------- ---------- ----------
 256**n's          65536's     256's       1's  

```
</center>

So we have a representation. In Python we could use the `list` data structure to store this: $[a_n, ..., a_2, a_1, a_0]$. 

The decimal representation of 518 would have $a_2 = 5$, $a_1 = 1$, $a_0 = 8$ and leaving us with a representation of `[5, 1, 8]`. 

The base 256 respresentation of 518 would have $a_1 = 2$ and $a_0 = 6$ leaving us with a representation of `[2, 6]`

Let's update the `bites` constructor to use this representation:

In [72]:
class bites:
    
    def __init__(self, values):
        # a list of numbers x: 0 <= x < 256 
        self.values = values
        
    def __repr__(self):
        return "(cheating) " + repr(self.values)

    def to_int(self, byte_order):
        raise NotImplementedError()
    
    @classmethod
    def from_int(cls, n, length, byte_order):
        raise NotImplementedError()

Now we just need an algorithm to go back and forth between `bites` and `int`. Let's start with the `bites` -> `int` direction.

# Implement `bites.to_int(places)` in FIXME steps

Here's a number we will be working with throughout this lesson:

In [73]:
N = 92837365

Here is the list representation of the decimals / base-10 expansion of this number as discussed above:

In [74]:
PLACES = [9, 2, 8, 3, 7, 3, 6, 5]

## Step 1: `decimal_places_to_int(places)`

Your first exercise is to write a `decimal_places_to_int(places)` function. It's correct if `decimal_places_to_int(PLACES) == N`.

This is a tricky exercise so take your time with it!

In [75]:
def decimal_places_to_int(places):
    number = 0
    places_reverse = places[::-1]
    for i in range(len(places_reverse)):
        number += places_reverse[i] * 10**i
    return number

In [76]:
def test():
    assert decimal_places_to_int(PLACES) == N
    print("Test passed!")

test()

Test passed!


## Step 2: `base_256_places_to_int(places)`

Now do it with base-256.

It's correct if `base_256_places_to_int([5, 136, 149, 245]) == N`

This one's quite a bit easier!

In [77]:
def base_256_places_to_int(places):
    result = 0
    places_reverse = places[::-1]
    for i in range(len(places_reverse)):
        result += places_reverse[i] * 256**i
    return result

In [78]:
def test():
    assert base_256_places_to_int([5, 136, 149, 245]) == N
    print("Test passed!")

test()

Test passed!


## Step 3: `places_to_int(places, base)`

Modify `base_256_places_to_int` so that it accepts arbitrary bases (e.g. 10 or 256):

In [79]:
def places_to_int(places, base):
    result = 0
    places_reverse = places[::-1]
    for i in range(len(places_reverse)):
        result += places_reverse[i] * base**i
    return result

In [80]:
def test():
    assert places_to_int([5, 136, 149, 245], 256) == N
    assert places_to_int([int(x, 16) for x in hex(N)[2:]], 16) == N
    assert places_to_int([int(x, 2) for x in bin(N)[2:]], 2) == N
    print("Test passed!")

test()

Test passed!


## Step 4: `places_to_int(places, base, byte_order)`

This is how we've been choosing to represent the number `92837365` in base-256: `[5, 136, 149, 245]`

The larger places on the left, smaller places to the right. Just like familiar decimal nubmers

```
    5     136     149     245
-----   -----   -----   -----
264^3   264^2   264^1   264^0
```

But isn't this choice completely arbitrary? Why not do the opposite: smaller places to the left, larger places to the right? 

```
  245     149     136       5
-----   -----   -----   -----
264^0   264^1   264^2   264^3
```

These choices are completely arbitrary! Different areas of computer science prefer one way or the other.

The order we've been using up to this point -- big places to the left, little places to the right -- is called **"Big Endian"**. This is preferred in network programming.

The second one -- little places to the left, big places to the right -- is called **"Little Endian"**. This is how computers usually store information internally.

Even within Bitcoin, Satoshi didn't choose one way or the other. He generally preferred "little endian" but he encodes IP addresses using "big endian". What a mess!

You can find an interesting discussion of "endianness" [here](https://bitcoin.stackexchange.com/questions/2063/why-does-the-bitcoin-protocol-use-the-little-endian-notation) and a nice YouTube video [here](https://www.youtube.com/watch?v=seZLUbgbB7Y)


##### Exercise: add another parameter to our function so that it can handle "Little Endian" byte order. 

* If `byte_order` is `"little"` you just need to reverse the list:
* Raise a `ValueError` if `byte_order` isn't `"big"` or `"little"`

In [81]:
from utils import assert_raises

def places_to_int(places, base, byte_order):
    if byte_order != "big" and byte_order != "little":
        raise ValueError("byte_order is not big or little")
    if byte_order == "big":
        places_converted = places[::-1]
    else:
        places_converted = places
    result = 0
    for i in range(len(places_converted)):
        result += places_converted[i] * base**i
    return result
    

In [82]:
def test():
    assert places_to_int([5, 136, 149, 245], 256, 'big') == N
    assert places_to_int([5, 136, 149, 245][::-1], 256, 'little') == N
    
    assert places_to_int([int(x, 16) for x in hex(N)[2:]], 16, 'big') == N
    assert places_to_int([int(x, 16) for x in hex(N)[2:]][::-1], 16, 'little') == N

    assert_raises(places_to_int, [], 10, 'foo')
    
    print("Test passes")
    
test()

Successfully observed expected error: 'byte_order is not big or little'
Test passes


## Step 5: `bites.to_int(base, byte_order)`

Bring it all together. Implement the `bites.from_int` classmethod.

Hint: return `cls(values)` ...

In [83]:
class bites:
    
    def __init__(self, values):
        self.values = values
        
    def __repr__(self):
        return "(cheating) " + repr(self.values)
    
    def to_int(self, byte_order):
        if byte_order != "big" and byte_order != "little":
            raise ValueError("byte_order is not big or little")
        if byte_order == "big":
            places_converted = self.values[::-1]
        else:
            places_converted = self.values
        result = 0
        for i in range(len(places_converted)):
            result += places_converted[i] * 256**i
        return int(result)
    
    @classmethod
    def from_int(cls, n, length, byte_order):
        raise NotImplementedError()

In [84]:
def test():
    places = [5, 136, 149, 245]
    assert bites(places).to_int('big') == N
    assert bites(places[::-1]).to_int('little') == N

test()

You've now completely implemented the functionality provided by `int.from_bytes`. Nice work! Now to the next method ...

# Implement `bites.from_int(...)`

`bites.to_int` handled `bites`-to-`int` conversions. Now let's implement a `bites.from_int` classmethod to do `int`-to-`bites` conversions. 


### `classmethod`

If you want a refresher on classmethods, check out [this video](https://www.youtube.com/watch?v=rq8cL2XMM5M&t=18s). They're useful when you want to create an instance of a class from data which doesn't fit in the class' `__init__` [constructor](https://stackoverflow.com/a/8986413/2542016). In our case, we can't call `bites(92837365)` because the bites constructor expects a `list` and not an `int`. We could put some fancy logic in our `bites.__init__` to check if the argument is an integer and convert if it is -- but that is frowned upon because it makes the `__init__` function hard to read. 

Instead, we will write a `bites.from_int` classmethod which will allow us to create `bites` instances directly from `int`s: 

> ```some_bites = bites.from_int(some_int)```

It will translate the `int` into a `list` and then pass that to the constructor `bites.__init__`. Our constructor stays nice and clean and we can still do type conversions!

### Division

`bytes.to_int` used multiplication: we started with a list of values, multified them by coefficients according to some rules, and added up the result.

`bytes.from_int` will use division. We will start with a number and divide off the base-256 places one at a time. It will be like dealing from a deck of cards ....

![ChessUrl](https://tenor.com/view/ref-with-yellow-cards-fifa18-pose-throw-cards-gif-12023802.gif "chess")


Division can be confusing. [Watch the next 10 seconds of this childrens' video for a refresher](https://youtu.be/KGMf314LUc0?t=76).

To compute $\frac{x}{y}$, we just just take groups of `y` from `x` until some number less than `y` is left. What's left is called the "remainder". 

Here's some notation and vocab review to jog your memory:

$$
dividend ÷ divisor = quotient + \frac{remainder}{divisor}
$$

$$
\require{enclose}
\begin{array}{rll}
    quotient && \hbox{+ remainder} \\[-3pt]
   divisor \enclose{longdiv}{dividend}\kern-.2ex \\[-3pt]
  \end{array}
$$

$$
\frac{dividend}{divisor} = quotient + \frac{remainder}{divisor} \hspace{1cm} \hbox{where remainder is < divisor} 
$$

### Division in Python

Python has 3 primary division operators:

`/` is floating point division

`//` is floor division

`%` is the "modulus" (remainder after division)

In [85]:
# Floating point division
# Returns quotient + remainder / divisor as a float

5 / 2

2.5

In [86]:
# Floor division 
# Returns the quotient, discards the remainder

5 // 2

2

In [87]:
# Modulus
# Returns the remainder

5 % 2

1

Let's attempt to use these division operators to deconstruct the hex representation of N:

In [88]:
hex(N)

'0x58895f5'

In [89]:
# Modulus gives us the right-most place
hex(N % 16)

'0x5'

In [90]:
# Floor division gives us the rest of the places
hex(N // 16)

'0x58895f'

In [91]:
# Combining // and % can traverse the hex representation
# This computs the 2nd place

hex(N // 16 % 16)

'0xf'

In [92]:
# How they're related
N == 16 * (N // 16) + (N % 16)

True

Now you know how to compute the 1's place ($16^0$) of the hexidecimal representation of N.

##### Exercise: Compute the 1's place ($256^0$) of the base-256 representation of N

In [93]:
N

92837365

In [94]:
256 * (N // 256) + (N % 256)

92837365

In [95]:
N % 256

245

In [96]:
def get_ones_place():
    return N % 256

In [97]:
def test_get_ones_place():
    assert get_ones_place() == 245

test_get_ones_place()

Now you know how to compute the 1's place ($16^0$) of the hexidecimal and base-256 representations of N.

##### Exercise: Compute the 65536's place ($256^2$) using only the `//` and `%` operators?

In [98]:
def get_65536s_place():
    return N // 256**2 % 256

In [99]:
get_65536s_place()

136

In [100]:
def test_get_65536s_place():
    return get_65536s_place() == 136

## Step 1:  `int_to_256_places(n)`

Use the techniques above to compute base-256 representation of numbers of the form discussed in `bites.to_int` section above.

If you can't get it, check out the table 3 cells down ....

In [101]:
156 // 10**0 % 10

6

In [102]:
260 // 256**2

0

In [103]:
N % 256

245

In [104]:
def int_to_base_256_places(n):
    results = []
    i = 0
    while n // 256**i > 0:
        results.append(n // 256**i % 256)
        i += 1
    return results[::-1]

In [105]:
int_to_base_256_places(N)

[5, 136, 149, 245]

In [106]:
def test():
    assert int_to_base_256_places(N) == [5, 136, 149, 245]
    
test()

Here's how my solution works:

**step**|**n**|**places**|**explanation**
:-----:|:-----:|:-----:|:-----:
 |92837365|[]|initial
%|92837365|[245]|92837365 % 256 == 245
//|362645|[245]|92837365 // 256 == 362645
%|362645|[149, 245]|362645 % 256 == 149 ; insert at index 0
//|1416|[149,  245]|362645 // 256 == 1416
%|1416|[136, 149, 245]|1416 % 256 == 136 ; insert at index 0
//|5|[136, 149, 245]|1416 // 256 == 5
%|5|[5, 136, 149, 245]|5 % 256 == 5 ; insert at index 0
//|0|[5, 136, 149, 245]|5 // 256 == 0
...|...|...|loop terminates

See how it's like dealing cards? With every loop we grab off the modulus and wipe out 1 factor of 256 from the polynomial expansion of `n` so that the next time we look for the modulus we'll get the next place. (FIXME: formatting below sucks)

$$ a_n * 256^n + ... + a_1 * 256^1 + a_{0} * 256^0  \mod 256 = a_0 $$
$$ a_n * 256^n + ... + a_1 * 256^1 + a_{0} * 256^0  // 256 = a_{n-1} * 256^{n-1} + ... + a_1 * 256^0 + 0  $$
$$  a_{n-1} * 256^{n-1} + ... + a_1 * 256^0  \mod 256 = a_1 $$
$$ ... $$

## Step 2: `int_to_places(n, base)`

Now add a second parameter to the function and rename it to `int_to_places(n, base)` so that it works with any base (e.g. 10 or 256).

Test it against the binary, octal, and hex representations we say above. For example, we saw `hex(92837365)` was `0x58895f5`, so as a list it should be `[5, 8, 8, 9, 5, 15, 5]` (hex `f` equals decimal `15`).

In [107]:
def int_to_places(n, base):
    results = []
    i = 0
    while n // base**i > 0:
        results.append(n // base**i % base)
        i += 1
    return results[::-1]

print(int_to_places(N, 16))  # is this what you expect?

[5, 8, 8, 9, 5, 15, 5]


In [108]:
def test():
    assert int_to_places(N, 256) == [5, 136, 149, 245]
    assert int_to_places(N, 16) == [int(x, 16) for x in hex(N)[2:]]
    assert int_to_places(N, 2) == [int(x, 2) for x in bin(N)[2:]]
    
test()

## Step 3: `int_to_places(n, base, length)`

Remember how every field in the protocol docs' "Message Structure" table has a `length` attribute?

![image](../images/message-structure.png)

We need to be able to support that, too. We should be able to say `int_to_places(1, 4)` and get [0, 0, 0, 1]. This feature helps us interpret and produce n-byte integer fields we encounter in the Bitcoin protocol.

Raise a `ValueError` if `n` doesn't fit in that many `bites` using the given `base`

In [111]:
def int_to_places(n, base, length):
    if n // base**length != 0:
        raise ValueError("length too short for value n")
    results = []
    for i in range(length):
        results.append(n // base**i % base)
    return results[::-1]

In [113]:
int_to_places(N, 256, 10)

[0, 0, 0, 0, 0, 0, 5, 136, 149, 245]

In [112]:
def test():
    assert int_to_places(N, 256, 10) == [0] * 6 + [5, 136, 149, 245]
    
    vals = [int(x, 16) for x in hex(N)[2:]]
    zeros = [0] * (20 - len(vals))
    assert int_to_places(N, 16, 20) == zeros + vals

    # You need 2 places to encode 16 in hex (0x10)
    assert_raises(int_to_places, 16, 16, 1)

test()

Successfully observed expected error: 'length too short for value n'


## Step 4:  `int_to_places(n, base, length, byte_order)`

Add another parameter to our function so that it can handle "Little Endian" byte order. If the `byte_order` isn't `"little"` or `"big"` then raise a `ValueError`.

In [118]:
def int_to_places(n, base, length, byte_order):
    if byte_order != "big" and byte_order != "little":
        raise ValueError("no valid encoding order")
    if n // base**length != 0:
        raise ValueError("length too short for value n")
    results = []
    for i in range(length):
        results.append(n // base**i % base)
    if byte_order == "big":
        results = results[::-1]
    return results

In [119]:
int_to_places(N, 256, 4, 'big')

[5, 136, 149, 245]

In [120]:
def test():
    vals = [5, 136, 149, 245]
    assert int_to_places(N, 256, len(vals), 'big') == vals
    assert int_to_places(N, 256, len(vals), 'little') == vals[::-1]

    vals = [int(x, 16) for x in hex(N)[2:]]
    assert int_to_places(N, 16, len(vals), 'big') == vals
    assert int_to_places(N, 16, len(vals), 'little') == vals[::-1]

    assert_raises(int_to_places, 1, 10, 1, 'dog')

test()

Successfully observed expected error: 'no valid encoding order'


## Step 5:  `bites.from_int(n, base, length, byte_order)`


Let's put it all together. Fill out the `from_int` method below and get the tests to pass

In [141]:
class bites:
    
    def __init__(self, values):
        self.values = values
        
    def __repr__(self):
        return "(cheating) " + repr(self.values)
    
    def to_int(self, byte_order):
        return places_to_int(self.values, 256, byte_order)
    
    @classmethod
    def from_int(cls, n, length, byte_order):
        if byte_order != "big" and byte_order != "little":
            raise ValueError("no valid encoding order")
        if n // 256**length != 0:
            raise ValueError("length too short for value n")
        results = []
        for i in range(length):
            results.append(n // 256**i % 256)
        if byte_order == "big":
            results = results[::-1]
        return cls(results)

In [142]:
bites.from_int(N, 4, 'big').values

[5, 136, 149, 245]

In [143]:
def test():
    vals = [5, 136, 149, 245]
    assert bites.from_int(N, len(vals), 'big').values == vals
    assert bites.from_int(N, len(vals), 'little').values == vals[::-1]
    
    # round trip
    assert bites.from_int(N, 4, 'big').to_int('big') == N

test()

# `bites.__repr__`

The representations of Python objects are determined by `.__repr__()` methods.

Let's see `bytes.__repr__` in action:

In [144]:
for i in range(256):
    print(i, "->", bytes([i]))

0 -> b'\x00'
1 -> b'\x01'
2 -> b'\x02'
3 -> b'\x03'
4 -> b'\x04'
5 -> b'\x05'
6 -> b'\x06'
7 -> b'\x07'
8 -> b'\x08'
9 -> b'\t'
10 -> b'\n'
11 -> b'\x0b'
12 -> b'\x0c'
13 -> b'\r'
14 -> b'\x0e'
15 -> b'\x0f'
16 -> b'\x10'
17 -> b'\x11'
18 -> b'\x12'
19 -> b'\x13'
20 -> b'\x14'
21 -> b'\x15'
22 -> b'\x16'
23 -> b'\x17'
24 -> b'\x18'
25 -> b'\x19'
26 -> b'\x1a'
27 -> b'\x1b'
28 -> b'\x1c'
29 -> b'\x1d'
30 -> b'\x1e'
31 -> b'\x1f'
32 -> b' '
33 -> b'!'
34 -> b'"'
35 -> b'#'
36 -> b'$'
37 -> b'%'
38 -> b'&'
39 -> b"'"
40 -> b'('
41 -> b')'
42 -> b'*'
43 -> b'+'
44 -> b','
45 -> b'-'
46 -> b'.'
47 -> b'/'
48 -> b'0'
49 -> b'1'
50 -> b'2'
51 -> b'3'
52 -> b'4'
53 -> b'5'
54 -> b'6'
55 -> b'7'
56 -> b'8'
57 -> b'9'
58 -> b':'
59 -> b';'
60 -> b'<'
61 -> b'='
62 -> b'>'
63 -> b'?'
64 -> b'@'
65 -> b'A'
66 -> b'B'
67 -> b'C'
68 -> b'D'
69 -> b'E'
70 -> b'F'
71 -> b'G'
72 -> b'H'
73 -> b'I'
74 -> b'J'
75 -> b'K'
76 -> b'L'
77 -> b'M'
78 -> b'N'
79 -> b'O'
80 -> b'P'
81 -> b'Q'
82 -> b'R'
83 -> b

I want you to implement a function that can print `bites` instances in the same way. To assist with this one I'm going to give you a list of character codes that have special meaning to `bytes`.

Below is a dictionary containing an `int -> ascii character` mapping of all numbers in 0 <= x < 256 with special meaning to `bytes`.

In [145]:
from utils import special_chars

print(special_chars)

{9: '\t', 10: '\n', 13: '\r', 32: ' ', 33: '!', 34: '"', 35: '#', 36: '$', 37: '%', 38: '&', 39: "'", 40: '(', 41: ')', 42: '*', 43: '+', 44: ',', 45: '-', 46: '.', 47: '/', 48: '0', 49: '1', 50: '2', 51: '3', 52: '4', 53: '5', 54: '6', 55: '7', 56: '8', 57: '9', 58: ':', 59: ';', 60: '<', 61: '=', 62: '>', 63: '?', 64: '@', 65: 'A', 66: 'B', 67: 'C', 68: 'D', 69: 'E', 70: 'F', 71: 'G', 72: 'H', 73: 'I', 74: 'J', 75: 'K', 76: 'L', 77: 'M', 78: 'N', 79: 'O', 80: 'P', 81: 'Q', 82: 'R', 83: 'S', 84: 'T', 85: 'U', 86: 'V', 87: 'W', 88: 'X', 89: 'Y', 90: 'Z', 91: '[', 92: '\\', 93: ']', 94: '^', 95: '_', 96: '`', 97: 'a', 98: 'b', 99: 'c', 100: 'd', 101: 'e', 102: 'f', 103: 'g', 104: 'h', 105: 'i', 106: 'j', 107: 'k', 108: 'l', 109: 'm', 110: 'n', 111: 'o', 112: 'p', 113: 'q', 114: 'r', 115: 's', 116: 't', 117: 'u', 118: 'v', 119: 'w', 120: 'x', 121: 'y', 122: 'z', 123: '{', 124: '|', 125: '}', 126: '~'}


Any value left unassigned by that dictionary should be converted into "\x" + hexidecimal representation. For example the number 150 is ontside the dictionary. It would therefore be presented as `"\x96"` because it's hexidecimal representation is `96`.

### Exercise: Implement a  `represent` function that works exactly like `bytes.__repr__` but with strings

Here's how it should work:

`represent(bytes([145, 22, 75, 152, 83])) -> "\x91\x16K\x98S"`

Hint: to put a `\` in a string you need to escape like `\\`. So a newline `\n` for example would need to be input as `\\n`.

In [146]:
def represent(b):
    result = ""
    for n in b.values:
        if n in special_chars:
            result += special_chars[n]
        else:
            result += '\\x' + hex(n)[2:]
    return result

# How does it look?
represent(bites([145, 22, 75, 152, 83]))

'\\x91\\x16K\\x98S'

In [147]:
def test():
    assert represent(bites([145, 22, 75, 152, 83])) == "\\x91\\x16K\\x98S"

test()

# Put it all together (with some help from our friends)

I'm going to add 2 methods that our Lesson 1 code requires: `.strip` and `__eq__`. To simplify things I just convert to `bytes` and have it do all the work. I'll explain these after the exercise ...

Go ahead an copy your solution from the last exercise into place as `bites.__repr__`:

In [150]:
class bites:
    
    def __init__(self, values):
        self.values = values
    
    def __eq__(self, other):
        return self.values == other.values
    
    def __repr__(self):
        result = ""
        for n in self.values:
            if n in special_chars:
                result += special_chars[n]
            else:
                result += '\\x' + hex(n)[2:]
        return result
    
    def to_int(self, byte_order):
        return places_to_int(self.values, 256, byte_order)
    
    @classmethod
    def from_int(cls, n, length, byte_order):
        places = int_to_places(n, 256, length, byte_order)
        return cls(places)

    def strip(self, pattern):
        return bites(list(bytes(self.values).strip(pattern)))

# How does it look?
bites([145, 22, 75, 152, 83])

\x91\x16K\x98S

In [151]:
def test():
    assert repr(bites([145, 22, 75, 152, 83])) == "\\x91\\x16K\\x98S"

test()

### `bites.__eq__`

needed for  magic bytes comparisons

In [152]:
bites([0xf9, 0xbe, 0xb4, 0xd9]) == bites([0xF9, 0xBE, 0xB4, 0xD9])

True

### `bites.strip()` needed for reading commands

In [153]:
b = bites(list(b"version\x00\x00\x00\x00\x00"))

print("unstripped:", b)
print("stripped:", b.strip(b"\x00"))

unstripped: version\x0\x0\x0\x0\x0
stripped: version


### `BiteStream`

This class turns streams of `bytes` into streams of `bites`

In [158]:
class BitesStream:

    def __init__(self, stream):
        self.stream = stream

    def read(self, n):
        return bites(list(self.stream.read(n)))
        
    def __getattr__(self, name):
        return getattr(self.f, name)

### Hashing `bites`

`hashlib.sha256` requires inputs to the "Buffer API" which we won't bother with (you've got to implement it in C ...)

In [166]:
from hashlib import sha256

def compute_checksum(b):
    hashed = sha256(sha256(bytes(b.values)).digest()).digest()
    checksum = hashed[:4]
    return b.__class__(list(checksum))

# Reading Bitcoin Messages From `bites`

A couple small tweeks to make our `NetworkEnvelope` class developed in Lesson 1 work with `bites` instead of `bytes`

In [167]:
NETWORK_MAGIC = bites([0xf9, 0xbe, 0xb4, 0xd9])

class NetworkEnvelope:

    def __init__(self, command, payload):
        self.command = command
        self.payload = payload

    @classmethod
    def from_stream(cls, stream):
        magic = stream.read(4)
        if magic != NETWORK_MAGIC:
            raise ValueError('Network magic is wrong')

        command = stream.read(12).strip(b"\x00")
        payload_length = stream.read(4).to_int('little')
        checksum = stream.read(4)
        payload = stream.read(payload_length)
        
        if checksum != compute_checksum(payload):  # homework: make bites subscriptable ...
            raise RuntimeError("Checksums don't match")

        return cls(command, payload)

    def __repr__(self):
        return f"<Message command={self.command}>"

In [169]:
import socket

# magic "version" bytestring
VERSION = b'\xf9\xbe\xb4\xd9version\x00\x00\x00\x00\x00j\x00\x00\x00\x9b"\x8b\x9e\x7f\x11\x01\x00\x0f\x04\x00\x00\x00\x00\x00\x00\x93AU[\x00\x00\x00\x00\x0f\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0f\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00rV\xc5C\x9b:\xea\x89\x14/some-cool-software/\x01\x00\x00\x00\x01'

PEER_IP = "85.234.209.217"
PEER_PORT = 8333

sock = socket.socket()
sock.connect((PEER_IP, PEER_PORT))
stream = sock.makefile('rb')
bites_stream = BitesStream(stream)

# initiate the "version handshake"
sock.send(VERSION)

# receive their "version" response
msg = NetworkEnvelope.from_stream(bites_stream)

print(msg)
print(msg.payload)

<Message command=version>


TypeError: to_int() missing 1 required positional argument: 'byte_order'

In [173]:
msg.payload.values

[127,
 17,
 1,
 0,
 13,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 196,
 197,
 86,
 92,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 255,
 255,
 31,
 10,
 150,
 60,
 99,
 155,
 13,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 196,
 115,
 163,
 172,
 98,
 222,
 234,
 69,
 16,
 47,
 83,
 97,
 116,
 111,
 115,
 104,
 105,
 58,
 48,
 46,
 49,
 55,
 46,
 49,
 47,
 192,
 144,
 8,
 0,
 1]

In [171]:
msg.payload.to_int("big")

216903867539253148402020219200948781647469366125018095432120378233580509455105829032722192117376461819218565968186501081608266657376189678501107684288671247778699304325486666163181021414769848384952932992474607653542448130874302277719731242270721